In [15]:
import random
from hashlib import sha256




In [16]:
def coprime(a, b):
    while b != 0:
        a, b = b, a % b
    return a

In [17]:
def extended_gcd(aa, bb):
    r1, r2 = abs(aa), abs(bb)
    s1, s2, t1, t2 = 0, 1, 1, 0
    while r2:
        r1, (q, r2) = r2, divmod(r1, r2)
        s1, s2 = s2 - q*s1, s1
        t1, t2 = t2 - q*t1, t1
    return r1, s2 * (-1 if aa < 0 else 1), t2 * (-1 if bb < 0 else 1)

In [18]:
def modinv(a, m):
    g, x, y = extended_gcd(a, m)
    if g != 1:
        raise Exception('Modular inverse does not exist')
    return x % m 

In [19]:
def is_prime(num):
    if num == 2:
        return True
    if num < 2 or num % 2 == 0:
        return False
    for n in range(3, int(num**0.5)+2, 2):
        if num % n == 0:
            return False
    return True

In [26]:
def generate_keypair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError('Both numbers must be prime.')
    elif p == q:
        raise ValueError('p and q cannot be equal')

    n = p * q

    #Phi is the totient of n
    phi = (p-1) * (q-1)

    #Choose an integer e such that e and phi(n) are coprime
    e = random.randrange(1, phi)

    #Use Euclid's Algorithm to verify that e and phi(n) are comprime 
    g = coprime(e, phi)
  
    while g != 1:
        e = random.randrange(1, phi)
        g = coprime(e, phi)

    #Use Extended Euclid's Algorithm to generate the private key
    d = modinv(e, phi)

    #Return public and private keypair
    #Public key is (e, n) and private key is (d, n)
    return ((e, n), (d, n))

In [21]:
def encrypt(privatek, plaintext):
    #Unpack the key into it's components
    key, n = privatek

    #Convert each letter in the plaintext to numbers based on the character using a^b mod m
            
    numberRepr = [ord(char) for char in plaintext]
    print("Number representation before encryption: ", numberRepr)
    cipher = [pow(ord(char),key,n) for char in plaintext]
    
    #Return the array of bytes
    return cipher

In [22]:
def decrypt(publick, ciphertext):
    #Unpack the key into its components
    key, n = publick
       
    #Generate the plaintext based on the ciphertext and key using a^b mod m
    numberRepr = [pow(char, key, n) for char in ciphertext]
    plain = [chr(pow(char, key, n)) for char in ciphertext]

    print("Decrypted number representation is: ", numberRepr)
    
    #Return the array of bytes as a string
    return ''.join(plain)

In [23]:
def hashFunction(message):
    hashed = sha256(message.encode("UTF-8")).hexdigest()
    return hashed

In [24]:
def verify(receivedHashed, message):
    ourHashed = hashFunction(message)
    if receivedHashed == ourHashed:
        print("Verification successful: ", )
        print(receivedHashed, " = ", ourHashed)
    else:
        
        print("Verification failed")
        print(receivedHashed, " != ", ourHashed)

In [27]:
p = int(input("Enter a prime number (17, 19, 23, etc): "))
q = int(input("Enter another prime number (Not one you entered above): "))   
#p = 17
#q=23


print("Generating your public/private keypairs now . . .")
public, private = generate_keypair(p, q)

print("Your public key is ", public ," and your private key is ", private)
message = input("Enter a message to encrypt with your private key: ")
print("")

hashed = hashFunction(message)

print("Encrypting message with private key ", private ," . . .")
encrypted_msg = encrypt(private, hashed)   
print("Your encrypted hashed message is: ")
print(''.join(map(lambda x: str(x), encrypted_msg)))
#print(encrypted_msg)

print("")
print("Decrypting message with public key ", public ," . . .")

decrypted_msg = decrypt(public, encrypted_msg)
print("Your decrypted message is:")  
print(decrypted_msg)

print("")
print("Verification process . . .")
verify(decrypted_msg, message)

Enter a prime number (17, 19, 23, etc): 23
Enter another prime number (Not one you entered above): 11
Generating your public/private keypairs now . . .
Your public key is  (201, 253)  and your private key is  (81, 253)
Enter a message to encrypt with your private key: avinash

Encrypting message with private key  (81, 253)  . . .
Number representation before encryption:  [53, 52, 102, 98, 101, 57, 50, 51, 52, 56, 54, 97, 52, 100, 49, 54, 48, 50, 49, 53, 52, 50, 102, 52, 98, 98, 53, 102, 52, 98, 100, 55, 54, 100, 53, 56, 102, 51, 51, 101, 48, 97, 57, 57, 101, 48, 100, 53, 53, 49, 49, 54, 48, 97, 51, 53, 101, 100, 52, 51, 101, 101, 98, 55]
Your encrypted hashed message is: 
15282125416779722498166186428232104186246721041528722128545415221285423212118623215216621224924916724642797916724623215215210410418624642249152167232824916716754121

Decrypting message with public key  (201, 253)  . . .
Decrypted number representation is:  [53, 52, 102, 98, 101, 57, 50, 51, 52, 56, 54, 97, 52, 100, 49